In [ ]:
from pathlib import Path

In [12]:
def save_dataset_to_file(dataset, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for pair in dataset:
            short_caption, long_caption = pair
            file.write(short_caption + '\n')
            file.write(long_caption + '\n')




In [2]:
def load_data(file_path):
    # Initialize an empty list to store caption pairs
    caption_pairs = []

    # Read lines from the file
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Iterate through lines to extract short and long captions
    i = 0
    while i < len(lines):
        if lines[i].startswith('short caption:'):
            short_caption = "prompt: "+lines[i].replace('short caption:', '').strip()
            i += 1
            if i < len(lines) and lines[i].startswith('long caption:'):
                long_caption = "response: "+lines[i].replace('long caption:', '').strip()
                caption_pairs.append((short_caption, long_caption))
            else:
                # If the next line doesn't start with 'long caption:', skip the entry
                i += 1
        else:
            i += 1

    return caption_pairs

In [5]:
from sklearn.model_selection import train_test_split

file_path = "llama_formatted_data.jsonl"
data = load_data(file_path)

# Split the data into training and evaluation sets
train_data, eval_data = train_test_split(data, test_size=0.1, random_state=42)  # Adjust test_size as needed

len(eval_data)

In [13]:
train_file_path = "./data/captions/captions.txt"
eval_file_path = "./data/captions_dev/captions.txt"

# Save training dataset to file
save_dataset_to_file(train_data, train_file_path)

# Save evaluation dataset to file
save_dataset_to_file(eval_data, eval_file_path)

-If dataset is in jsonl format

In [9]:
import jsonlines
import random
import os
import gcsfs

fs=gcsfs.GCSFileSystem(project='Upstart Fast Collaboration')

# Set the path to your JSONL file
jsonl_file = "gs://upstartdatasetimages/llama_formatted_data.jsonl"

# Set the paths for the train and test files
train_file = './data/captions/captions.jsonl'
test_file = "./data/captions_dev/captions.jsonl"

os.makedirs(os.path.dirname(train_file), exist_ok=True)
os.makedirs(os.path.dirname(test_file), exist_ok=True)

# Set the ratio of data to be used for training (e.g., 80%)
train_ratio = 0.8

# Read the data from the JSONL file
data = []
with jsonlines.open(jsonl_file, 'r') as reader:
    for line in reader:
        data.append(line)

# Shuffle the data randomly
random.shuffle(data)

# Calculate the number of samples for the training set
num_train_samples = int(train_ratio * len(data))

# Split the data into train and test sets
train_data = data[:num_train_samples]
test_data = data[num_train_samples:]

# Write the train data to the train file
with jsonlines.open(train_file, 'w') as writer:
    for record in train_data:
        writer.write(record)

# Write the test data to the test file
with jsonlines.open(test_file, 'w') as writer:
    for record in test_data:
        writer.write(record)

print("Dataset split into train and test sets successfully.")


FileNotFoundError: [Errno 2] No such file or directory: 'gs://upstartdatasetimages/llama_formatted_data.jsonl'

In [2]:
pip install jsonlines

Note: you may need to restart the kernel to use updated packages.
